In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from pandas import DatetimeIndex
from datetime import datetime

import yfinance as yf

import statsmodels.api as sm
from statsmodels.formula.api import ols
from patsy import dmatrices
import wrds

In [2]:
data = pd.read_csv('daily_data.csv')
data = data[data['ceq'] >= 0]
data['datadate']= pd.to_datetime(data['datadate'])


# Choose Company Ticker
stocks = list(data.tic.unique())
Q1 = 0
while Q1 == 0:
    print('\n','Please input your target Company Ticker')
    target_ticker = input()
    if target_ticker in stocks:
        Q1 = 1
    else:
        print('Error: Invalid Stock')
        Q1 = 0

intervals = ['1d', '1wk', '1mo']
Q2 = 0
while Q2 == 0:
    print('Enter Interval: 1d - daily, 1wk - weekly, 1mo - monthly')
    interval = input()
    if interval in intervals:
        Q2 = 1
    else:
        print('Error: Invalid Interval')
        Q2 = 0

Q3 = 0
while Q3 == 0:
    print('Enter Start Date in yyyy-mm-dd format')
    q3a = 0
    while q3a == 0:
        try:
            start = input()
            datetime.strptime(start,'%Y-%m-%d')
            q3a = 1 
        except:
            q3a = 0
            print('Wrong Format - Enter again.')
    
    print('Enter End Date in yyyy-mm-dd format')
    q3b = 0
    while q3b == 0:
        try:
            end = input()
            datetime.strptime(end,'%Y-%m-%d')
            q3b = 1 
        except:
            q3b = 0
            print('Wrong Format - Enter again.')

    ticker_close = yf.download(target_ticker, start= start ,end = end, interval = interval)[['Adj Close']]
    if len(ticker_close) > 10:
        Q3 = 1
    else:
        print('Error: Insufficient Data')
        print('Please restart or enter new dates')
        Q3 = 0

print('...getting data','\n')

market = yf.download('SPY', start= start ,end = end, interval = interval)[['Adj Close']]
rf = yf.download('^IRX',start_date = '2021-10-20')['Close']
rf = round(float(rf[rf.index == max(rf.index)].values),4)

market_returns_daily = market.pct_change().dropna() + 1
yearly_market_returns = market_returns_daily.groupby([market_returns_daily.index.year]).prod()-1
market_return = float(yearly_market_returns.mean().values) * 100

target_data = pd.DataFrame(np.zeros((1,5)), columns = ['symbol','shortName','totalDebt','marketCap','beta'])

temp = yf.Ticker(target_ticker).info
for i in target_data.columns.values:
    target_data[i] = temp[i]
target_data['Debt/Equity'] = target_data['totalDebt']/target_data['marketCap']

# Getting Equity beta from SCL
return_df = ticker_close.pct_change().dropna()
return_df['market'] = market_returns_daily['Adj Close']
return_df['Ri_minus_Rf'] = return_df['Adj Close']
return_df['Rm_minus_Rf'] = return_df['market']

y, X = dmatrices('Ri_minus_Rf ~ Rm_minus_Rf', data=return_df, return_type='dataframe')
regression = sm.OLS(y, X).fit()
scl_equity_beta = round(float(regression.params[1]),5)


# COMPS

def get_query(stock, sDate, eDate):
    query = data[data['tic'] == stock]
    query = query[query['datadate'] >= sDate]
    query = query[query['datadate'] <= eDate]
    query['D/E_Ratio'] = (query['dlc'] + query['dltt'])/query['ceq']
    ratio = query['D/E_Ratio'].mean()
    query['residuals'] = (query['D/E_Ratio'] - ratio)**2
    return query


def get_comps(industry, value, sDate, eDate):
    comps = data[data['datadate'] >= sDate]
    comps = comps[comps['datadate'] <= eDate]
    comps = comps[comps['sich'] == industry]
    comps = comps.drop(columns=['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc',
                               'datafmt', 'curcd', 'csho', 'cshpri', 'dlc', 'dltt', 'costat',
                               'prcc_c', 'ggroup', 'sich', 'sic'])
    comps = comps.groupby('tic').agg('mean').reset_index()
    
    comps['residuals'] = (comps['ceq'] - value)**2
    comps = comps.sort_values(by='residuals')
    comps = comps[1:6]
    comps = list(comps.tic.unique())
    
    return comps


datetimeobject = datetime.strptime(start,'%Y-%m-%d')
sDate = datetimeobject.strftime('%m/%d/%Y')

datetimeobject = datetime.strptime(end,'%Y-%m-%d')
eDate = datetimeobject.strftime('%m/%d/%Y')

query = get_query(target_ticker, sDate, eDate)

value = query['ceq'].mean()
industry = query['sich'].mean()

comps_ticker = get_comps(industry, value, sDate, eDate)

print('These are the 5 closest competitions by Market Value', comps_ticker)

print('...getting data','\n')

comps_data = pd.DataFrame(np.zeros((len(comps_ticker),5)), columns = ['symbol','shortName','totalDebt','marketCap','beta']).fillna(0.00)

for i in range(len(comps_ticker)):
    temp = yf.Ticker(comps_ticker[i]).info
    for j in comps_data.columns.values:
        try:
            comps_data.loc[i,j] = temp[j]
        except:
            comps_data.loc[i,j] = 0 
              
comps_data['Debt/Equity'] = comps_data['totalDebt']/comps_data['marketCap']

# Enter comps D/E Assumption

print('Financing:  Enter Numbers')
print('1. Constant D')
print('2. Constant D/E','\n')

for i in range(len(comps_data)):
    print('For', comps_data.loc[i,'symbol'])
    t = int(input())
    if t == 1:
        f_type = 'Constant D'
    if t == 2:
        f_type = 'Constant D/E'
        
    comps_data.loc[i,'F_type'] = f_type
    
print('For', target_data.loc[0,'symbol'])
t= input()
if t == 1:
    f_type = 'Constant D'
if t == 2:
    f_type = 'Constant D/E'
target_data.loc[0,'F_type'] = f_type

print('Enter Corporate Tax Rate')

tax_rate = float(input())

print('\n', 'Current Risk Free Rate is ', rf, '\n\n')

for i in range(len(comps_data)):
    if comps_data.loc[i,'F_type'] == 'Constant D':
        comps_data.loc[i,'Unlevered Beta'] = (comps_data.loc[i,'beta'])/(1+(1-tax_rate/100)*comps_data.loc[i,'Debt/Equity'])
    if comps_data.loc[i,'F_type'] == 'Constant D/E':
        comps_data.loc[i,'Unlevered Beta'] = (comps_data.loc[i,'beta'])/(1+comps_data.loc[i,'Debt/Equity'])


average_comp_unlevered_beta = round(comps_data['Unlevered Beta'].mean(),4)

if target_data.loc[0,'F_type'] == 'Constant D':
    target_data['Beta_Levered_Comps']   = average_comp_unlevered_beta*(1 + target_data.loc[0,'Debt/Equity']*(1-tax_rate/100))
if target_data.loc[0,'F_type'] == 'Constant D/E':
    target_data['Beta_Levered_Comps']   = average_comp_unlevered_beta*(1 + target_data.loc[0,'Debt/Equity'])
target_data['Beta_SCL_Regression'] = scl_equity_beta
    
print(comps_data.to_string(),'\n\n')

print('Average Comps Unlevered Beta is ', average_comp_unlevered_beta,'\n\n')

print(target_data.to_string(),'\n\n')

cost_of_equity = rf + (market_return - rf)*float(target_data['Beta_Levered_Comps'].values)


print('SCL Equity Beta = ', round(scl_equity_beta,2))
print('Comps Equity Beta = ', round(float(target_data['Beta_Levered_Comps'].values),2))

print('Cost of Equity (using Comps)', round(cost_of_equity,4))

print('\n')

if round(scl_equity_beta,2)>round(float(target_data['Beta_Levered_Comps'].values),2):
    print('Buy the ',target_ticker, ' stock' )
elif round(scl_equity_beta,2)<round(float(target_data['Beta_Levered_Comps'].values),2):
    print('Sell the ',target_ticker, ' stock')
else:
    print('Hold the ',target_ticker, ' stock')


 Please input your target Company Ticker
MSFT
Enter Interval: 1d - daily, 1wk - weekly, 1mo - monthly
1d
Enter Start Date in yyyy-mm-dd format
2010-01-01
Enter End Date in yyyy-mm-dd format
2020-01-01
[*********************100%***********************]  1 of 1 completed
...getting data 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
These are the 5 closest competitions by Market Value ['ORCL', 'SAP', 'LBRDK', 'ATVI', 'ADBE']
...getting data 

Financing:  Enter Numbers
1. Constant D
2. Constant D/E 

For ORCL
1
For SAP
1
For LBRDK
2
For ATVI
2
For ADBE
1
For MSFT
1
Enter Corporate Tax Rate
21

 Current Risk Free Rate is  0.035 


  symbol                      shortName     totalDebt     marketCap      beta  Debt/Equity        F_type  Unlevered Beta
0   ORCL             Oracle Corporation  8.271800e+10  2.600833e+11  0.800000     0.318044    Constant D        0.639358
1    SAP                     